In [1]:
from pynq import Overlay
from pynq import PL

base = Overlay('/home/xilinx/jupyter_notebooks/weikai/dma_100M.bit', download=True)
ofl = open("asdf.txt",'w')

In [2]:
ofl.write(f"{base.ip_dict}")
ofl.write("\n\n\n\n")
ofl.write(f"{base.hierarchy_dict}")

2

In [3]:
import numpy as np
from pynq import allocate

fourK_byte = 3840 * 2160 * 3

input_buffer = allocate(shape=(fourK_byte,),dtype=np.uint8)
output_buffer = allocate(shape=(fourK_byte,),dtype=np.uint8)
print(hex(input_buffer.device_address))
print(hex(output_buffer.device_address))

0x60300000
0x61b00000


In [4]:
# initialize IO buffer
for i in range(fourK_byte):
    input_buffer[i] = 255
for i in range(fourK_byte):
    output_buffer[i] = 0

In [5]:
# start on dma passthrough 
from pynq import MMIO
# get the dma address and map it using MMIO
print(hex(base.ip_dict['axi_dma_0']['phys_addr']))
dma = MMIO(0xa0000000, 0x60)
# print(input_buffer.device_address)
# print(hex(input_buffer.device_address))
# print(hex(output_buffer.device_address))

0xa0000000


In [6]:
# Read the control logic of dma

print(hex(dma.read(0)))     #MM2S control
print(hex(dma.read(4)))     #MM2S status

print(hex(dma.read(0x30)))  #S2MM control
print(hex(dma.read(0x34)))  #S2MM status

0x10002
0x1
0x10002
0x1


In [7]:
# check the input buffer before transfer
for i in range(32):
    print(input_buffer[i])

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255


In [8]:
# Read from Input Buffer using MM2S
# MM2S control setting

# set run/stop bit to 1
dma.write(0, 1)
# set upper address
dma.write(0x1c, 0)
# set lower address
dma.write(0x18, input_buffer.device_address)
# set lenth in bytes (8*4)
dma.write(0x28, fourK_byte)

In [9]:
# check the output buffer before transfer
for i in range(32):
    print(output_buffer[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [10]:
# Write to Output Buffer using S2MM
# S2MM control setting

# set run/stop bit to 1
dma.write(0x30, 1)
# set upper address
dma.write(0x4c, 0)
# set lower address
dma.write(0x48, output_buffer.device_address)
# set lenth in bytes (4*4)
dma.write(0x58, fourK_byte)

In [11]:
# Read the control logic of dma

print(hex(dma.read(0)))     #MM2S control
print(hex(dma.read(4)))     #MM2S status

print(hex(dma.read(0x30)))  #S2MM control
print(hex(dma.read(0x34)))  #S2MM status

0x10003
0x1002
0x10003
0x1002


In [12]:
# check the output buffer after transfer
for i in range(32):
    print(output_buffer[i])

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255


In [13]:
import time


t1 = time.time()

for i in range(100):
    # set run/stop bit to 1
    dma.write(0, 1)
    # set upper address
    dma.write(0x1c, 0)
    # set lower address
    dma.write(0x18, input_buffer.device_address)
    # set lenth in bytes (8*4)
    dma.write(0x28, fourK_byte)

    # set run/stop bit to 1
    dma.write(0x30, 1)
    # set upper address
    dma.write(0x4c, 0)
    # set lower address
    dma.write(0x48, output_buffer.device_address)
    # set lenth in bytes (4*4)
    dma.write(0x58, fourK_byte)

    while hex(dma.read(0x34)) != "0x1002":
              pass
    

t2 = time.time()
          
print(hex(dma.read(0)))     #MM2S control
print(hex(dma.read(4)))     #MM2S status

print(hex(dma.read(0x30)))  #S2MM control
print(hex(dma.read(0x34)))  #S2MM status

print(100 / (t2-t1))


0x10003
0x1002
0x10003
0x1002
63.94393587409476


SyntaxError: incomplete input (147812726.py, line 1)

In [13]:
del input_buffer
del output_buffer